# It looks like PIMS is running into issues when reading some ome-tiffs

Let's see if tifffile can do better.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import importlib
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
    
import imaging_db.filestorage.s3_storage as s3_storage

/Users/jenny.folkesson/Code/imagingDB


In [2]:
import imaging_db.database.db_session as db_session

# Import all our tables
from imaging_db.database.base import Base
from imaging_db.database.file_global import FileGlobal
from imaging_db.database.frames_global import FramesGlobal
from imaging_db.database.frames import Frames
# This is the overall table containing the identifier and description
from imaging_db.database.dataset import DataSet


importlib.reload(db_session)
credentials_filename = "/Users/jenny.folkesson/Code/db_credentials.json"

In [3]:
with db_session.session_scope(credentials_filename) as session:
    print(Base.metadata.tables.keys())

dict_keys(['file_global', 'frames_global', 'frames', 'data_set'])


ISP-2018-06-08-15-45-00-0001 and ISP-2018-06-08-15-45-00-0002 are the same file, but one is uploaded with PIMS and the other with tifffile. Are they the same?

In [4]:
db_ops = db_session.DatabaseOperations(
    credentials_filename,
    dataset_serial="ISP-2018-06-08-15-45-00-0001",
)
global_meta, frames_meta = db_ops.get_frames_meta()
print(global_meta)

{'s3_dir': 'raw_frames/ISP-2018-06-08-15-45-00-0001', 'nbr_frames': 39, 'im_width': 2048, 'im_height': 2048, 'nbr_slices': 13, 'nbr_channels': 3, 'im_colors': 1, 'nbr_timepoints': 1, 'nbr_positions': 1, 'bit_depth': 'uint16'}


In [6]:
db_ops2 = db_session.DatabaseOperations(
    credentials_filename,
    dataset_serial="ISP-2018-06-08-15-45-00-0002",
)
global_meta2, frames_meta2 = db_ops2.get_frames_meta()
print(global_meta2)

{'s3_dir': 'raw_frames/ISP-2018-06-08-15-45-00-0002', 'nbr_frames': 39, 'im_width': 2048, 'im_height': 2048, 'nbr_slices': 13, 'nbr_channels': 3, 'im_colors': 1, 'nbr_timepoints': 1, 'nbr_positions': 1, 'bit_depth': 'uint16'}


Global metadata looks the same, let's check local meta

In [ ]:
dataset_identifier = "ISP-2018-06-08-15-45-00-0001"

first_frame = session.query(Frames) \
    .join(FramesGlobal) \
    .join(DataSet) \
    .filter(DataSet.dataset_serial == dataset_identifier) \
    .first()

s3_dir = first_frame.frames_global.s3_dir
print(s3_dir)
print(first_frame.)